In [3]:
from glob import glob
import numpy as np
import pandas as pd
import os

In [4]:
galDB = pd.read_pickle('../data/sgal_param_DB.pkl')
galDB = galDB.query('ids != 14713 and ids != 38126')
lzDB = galDB.query('hi_res_specz < 1.16')
mzDB = galDB.query('1.16 < hi_res_specz < 1.3')
hzDB = galDB.query('1.3 < hi_res_specz < 1.45')
uhzDB = galDB.query('1.45 < hi_res_specz')

metal=np.arange(0.002,0.031,0.001)
age=np.arange(.5,6.1,.1)
tau=[0,8.0, 8.3, 8.48, 8.6, 8.7, 8.78, 8.85, 8.9, 8.95, 9.0, 9.04, 9.08, 9.11, 9.15, 9.18, 9.2, 9.23, 9.26, 9.28,
     9.3, 9.32, 9.34, 9.36, 9.38, 9.4, 9.41, 9.43, 9.45, 9.46, 9.48]

In [3]:
for i in uhzDB.index:
    print  3900 * (1 + uhzDB['hi_res_specz'][i])


9804.6
10264.8
10120.5
10756.2
10151.7
10218.0
10182.9


In [4]:
len(uhzDB.index)

7

In [12]:
slrm_text= ["#!/bin/bash\n#SBATCH -J gt_fit-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o gfit{0}-%j.out\n\
#SBATCH -e gfit{0}-%j.err\n\nchmod a+x C_gt_3900_fit.py\n\n\
python C_gt_3900_fit.py '{1}' {2} &\npython C_gt_3900_fit.py '{3}' {4} &\n\
python C_gt_3900_fit.py '{5}' {6} &\n\
python C_gt_3900_fit.py '{7}' {8} &\n\
\nwait\n\nexit 0".format(1, uhzDB.gids[uhzDB.index[0]],uhzDB.hi_res_specz[uhzDB.index[0]],
                        uhzDB.gids[uhzDB.index[1]],uhzDB.hi_res_specz[uhzDB.index[1]],
                        uhzDB.gids[uhzDB.index[2]],uhzDB.hi_res_specz[uhzDB.index[2]],
                        uhzDB.gids[uhzDB.index[3]],uhzDB.hi_res_specz[uhzDB.index[3]],)]
        
np.savetxt('GT_fit_1.slrm', slrm_text, fmt='%s')

In [13]:
slrm_text= ["#!/bin/bash\n#SBATCH -J gt_fit-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o gfit{0}-%j.out\n\
#SBATCH -e gfit{0}-%j.err\n\nchmod a+x C_gt_3900_fit.py\n\n\
python C_gt_3900_fit.py '{1}' {2} &\npython C_gt_3900_fit.py '{3}' {4} &\n\
python C_gt_3900_fit.py '{5}' {6} &\n\
\nwait\n\nexit 0".format(2, uhzDB.gids[uhzDB.index[4]],uhzDB.hi_res_specz[uhzDB.index[4]],
                        uhzDB.gids[uhzDB.index[5]],uhzDB.hi_res_specz[uhzDB.index[5]],
                        uhzDB.gids[uhzDB.index[6]],uhzDB.hi_res_specz[uhzDB.index[6]])]

np.savetxt('GT_fit_2.slrm', slrm_text, fmt='%s')

In [17]:
slrm_text= ["#!/bin/bash\n#SBATCH -J no_Hb-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o no_Hb{0}-%j.out\n\
#SBATCH -e no_Hb{0}-%j.err\n\nchmod a+x C_no_Hb_fit.py\n\n\
python C_no_Hb_fit.py '{1}' {2} &\npython C_no_Hb_fit.py '{3}' {4} &\n\
python C_no_Hb_fit.py '{5}' {6} &\n\
\nwait\n\nexit 0".format(1, 'n21156',1.251,
                        's47677',1.112,
                        's39170',1.022)]

np.savetxt('No_hb.slrm', slrm_text, fmt='%s')

In [11]:
blist = glob('/Volumes/Vince_research/Extractions/Updated_Q_gals/stack_specs/*')
n_beams = glob('/Volumes/Vince_research/clear_q_beams/*')
nbeamid = [int(U.replace('.g102.A.fits','').split('_')[-1]) for U in n_beams]

In [12]:
slist = []
for i in blist:
    if int(os.path.basename(i).replace('.npy','')[1:]) in nbeamid:
        slist.append(os.path.basename(i).replace('.npy',''))

In [16]:
int(len(slist)/4) * 4

52

In [18]:
for i in range(int(len(slist)/4)):
    slrm_text= ["#!/bin/bash\n#SBATCH -J zfit-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o zfit{0}-%j.out\n\
#SBATCH -e zfit{0}-%j.err\n\nchmod a+x C_Specz_instruct.py\n\n\
python C_Specz_instruct.py '{1}' &\npython C_Specz_instruct.py '{2}' &\n\
python C_Specz_instruct.py '{3}' &\n\
python C_Specz_instruct.py '{4}' &\n\
\nwait\n\nexit 0".format(i + 1, slist[4*i + 0], slist[4*i + 1], 
                             slist[4*i + 2], slist[4*i + 3],)]

    np.savetxt('../cluster_scripts/rshiftfit_{0}.slrm'.format(i + 1), slrm_text, fmt='%s')

In [19]:
slrm_text= ["#!/bin/bash\n#SBATCH -J zfit-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o zfit{0}-%j.out\n\
#SBATCH -e zfit{0}-%j.err\n\nchmod a+x C_Specz_instruct.py\n\n\
python C_Specz_instruct.py '{1}' &\npython C_Specz_instruct.py '{2}' &\n\
python C_Specz_instruct.py '{3}' &\n\
\nwait\n\nexit 0".format(14, slist[-3], slist[-2], 
                             slist[-1],)]

np.savetxt('../cluster_scripts/rshiftfit_{0}.slrm'.format(14), slrm_text, fmt='%s')

In [2]:
galDB = pd.read_pickle('../data/upd_galDB.pkl')


In [16]:
for i in range(len(galDB.gids)):
    slrm_text= ["#!/bin/bash\n#SBATCH -J faz-{0}\n#SBATCH -p background\n#SBATCH --time=20:00:00\n#SBATCH --mem-per-cpu=64000\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=32000\n#SBATCH -o faz{0}-%j.out\n\
#SBATCH -e faz{0}-%j.err\n\nchmod a+x C_fit_full_analyze.py\n\npython C_fit_full_analyze.py '{1}' {2} {3} {4} '{5}' &\n\nwait\n\nexit 0".format(i + 1, 
                galDB.gids[galDB.index[i]],galDB.z_grism[galDB.index[i]],
                np.round(galDB.z_grism[galDB.index[i]] - 0.02,3), np.round(galDB.z_grism[galDB.index[i]] + 0.02,3),
                '{0}_full_fit'.format(galDB.gids[galDB.index[i]]))]

    np.savetxt('../cluster_scripts/fullfit_az_{0}.slrm'.format(i + 1), slrm_text, fmt='%s')

AttributeError: 'DataFrame' object has no attribute 'z_grism'

In [5]:
31*29

899

In [8]:
mp = [0.005,0.007,0.009,0.011,0.013,0.015,0.017,0.019,0.021,0.023,0.025,0.027]
ap = [3.2,2.2,1.0,3.8,2.7,1.4,4.4,3.2,1.9,5.2,3.6,2.0]
sn = [2,4,8,12,16]

gals = ['s39631', 'n21156', 's40862', 's40223'] 

red = [1.057, 1.25, 1.328, 1.593]

In [17]:
for i in range(len(gals)):
    for ii in range(len(mp)):
        slrm_text= ["#!/bin/bash\n#SBATCH -J mcfit-{0}\n#SBATCH -p background\n#SBATCH --time=20:00:00\n#SBATCH --mem-per-cpu=64000\n#SBATCH -o mcfit{0}-%j.out\n#SBATCH -e mcfit{0}-%j.err\n\nchmod a+x C_MC_fit_instruct.py\n\
\npython C_MC_fit_instruct.py '{1}' {2} {3} {4} {5} {6} '{7}' '{8}' &\n\nwait\n\nexit 0".format(i*len(mp) + ii + 1, 
                    gals[i], red[i], np.round(red[i] - 0.02,3), np.round(red[i] + 0.02,3), mp[ii], ap[ii], 
                    '{0}_full_fit'.format(gals[i]), 'MC_{0}_m{1}_a{2}'.format(gals[i],mp[ii],ap[ii]))]

        np.savetxt('../cluster_scripts/MC_fit_{0}.slrm'.format(i*len(mp) + ii + 1), slrm_text, fmt='%s')